Guide : https://www.mlq.ai/deep-reinforcement-learning-for-trading-with-tensorflow-2-0/

## Stage 1: Installing dependencies and environment setup


In [ ]:
!pip install tensorflow-gpu==2.0.0rc0

     |████████████████████████████████| 380.5 MB 40 kB/s 
     |████████████████████████████████| 4.3 MB 42.1 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 501 kB 55.1 MB/s 


In [ ]:
!pip install pandas-datareader

## Stage 2: Importing project dependencies

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [ ]:
tf.__version__

'2.0.0-rc0'

## Stage 3: Building the AI Trader network

In [ ]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    # Define hyperparamaters
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
        
    # Call a function  to build a model trought this class constructor
    # More parameters could be ustilized to programaticaly define network size (layers and neurons)
    self.model = self.model_builder()
    
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))    
    return model
  
  
  
  
  # Trade function that takes state as an input and returns an action 
  # to perform in perticular state 
  def trade(self, state):
    
    # Should we perform a renadom generated action or action defined in model?
    
    # If value from our random generator is smaller or equal to our epsilon 
    #     then we will retun a random action from action_space [0-3)
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    # If our random is greater than epsilon then we will use model to perform action
    actions = self.model.predict(state)
    # return only a one number defining an action (#Stay - 0 , Buy - 1, Sell - 2) 
    #    that has maximum probability
    return np.argmax(actions[0])
  
  
  
  def batch_train(self, batch_size):
    
    batch = []
    
    # Iterrate in momory, we do not want to randolmy select data as we are dealing with 
    #    time constraint data. We will always sample from the end of memory size of bath
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # insert data from memory to batch      
      batch.append(self.memory[i])
    
    
    # Iterate trought batch of data and train the model for each sample from batch
    # Order of variables in for loop is important
    for state, action, reward, next_state, done in batch:
      # Reward if agent is in terminal state
      reward = reward
      # Check that agent is not in terminal state
      # If not in terminal state calculate reward for actions that could be played
      if not done:
        # Discounted total reward:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])        
      # Target variable that is predicted by the model (action)
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    # We will decrease epsilon parameter that is 1 as defined in __init__  so
    #    so we can stop performing random actions at some point
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Stage 4: Dataset preprocessing

### Defining helper functions

#### Sigmoid

In [ ]:
# Usually used at the end of a network for binary classifictation
# It changes range of input to scale of [0,1]
# So we can normalize input data for comparision day by day if they are on different scale
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Price format function

In [ ]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Dataset loader

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 18.0 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=05858af97cde93d021d8dea7f42ff55d15d9b5693cb4f2f7b439881c2815fb77
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import yfinance as yf
import numpy as np

def dataset_loader(stock_name):
  try:
    
    date1="2020-07-27"
    date2="2021-07-28"
    data = yf.download(tickers=stock_name, start=date1, end=date2, interval='1d' )
    close = data['Close']
    return close
  except Exception as e:
      print(e)


# Check the data gathered with pandas data_reader:
stock_name= "AAPL"
dataset = dataset_loader(stock_name)
print("Data set top rows:", "\n" ,dataset.head())

print("Test some cutting with pandas")
print("Start date: ", str(dataset.index[0]).split()[0])
print("End date: ", str(dataset.index[-1]).split()[0])

[*********************100%***********************]  1 of 1 completed
Data set top rows: 
 Date
2020-07-27     94.809998
2020-07-28     93.252502
2020-07-29     95.040001
2020-07-30     96.190002
2020-07-31    106.260002
Name: Close, dtype: float64
Test some cutting with pandas
Start date:  2020-07-27
End date:  2021-07-27


In [ ]:
"""def dataset_loader(stock_name, web_data_source="yahoo"):
  
  #Use pandas data reader for reading stock data from warious sources like "yahoo", "google"
  dataset = data_reader.DataReader(name=stock_name, data_source="yahoo")
   
  # Get start and end time to variables from dataset
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  # Model will use "Close" column for training 
  close = dataset['Close']

  return close"""

### State creator

In [ ]:
# Data -> dataset to predict from, gathered by data:loader()
# Timestep -> Day in the dataset that we want to predict for [0:datalength]
# window_suze -> how many days in past we want to use to predict current status[1:datalength]
#         Try different setup to see what creates best fit
def state_creator(data, timestep, window_size):
  
  # starting day of our state
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    # Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  # Iterate trough whole windowed_data minus current state (-1)
  for i in range(window_size - 1):
    # Normalize the difference from current day and the next day
    # Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Loading a dataset

In [ ]:
# Tage data for Apple
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%***********************]  1 of 1 completed


In [ ]:
data

Date
2020-07-27     94.809998
2020-07-28     93.252502
2020-07-29     95.040001
2020-07-30     96.190002
2020-07-31    106.260002
                 ...    
2021-07-21    145.399994
2021-07-22    146.800003
2021-07-23    148.559998
2021-07-26    148.990005
2021-07-27    146.770004
Name: Close, Length: 253, dtype: float64

## Stage 5: Training the AI Trader

### Setting hyper parameters

In [ ]:
window_size = 10
episodes = 1000 # same as epoch

batch_size = 32
data_samples = len(data) - 1 # discard last value, that we will predict on

### Defining the Trader model

In [ ]:
trader = AI_Trader(window_size)

In [ ]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training loop

In [ ]:
for episode in range(1, episodes + 1):
  
  # To keep track of training process
  # .format populates {} with variables in .format(x,y)
  print("Episode: {}/{}".format(episode, episodes))
  
  # Create state
  # second parameter is timestep = 0
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  # Empty inventory before starting episode
  trader.inventory = []
  
  # One timestep is one day so number of timesteps we have represent data we have
  # tqdm is used for visualization
  for t in tqdm(range(data_samples)):
    
    # First we will access action that is going to be taken by model 
    action = trader.trade(state)
    
    # Use action to get to next state(t+)
    next_state = state_creator(data, t+1, window_size + 1)
    # As we did not calculate anything up to this point reward is 0
    reward = 0
    
    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    # To sell we need to have something in inventory  
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)
      
      # If we gain money (current price - buy price) we have reward 
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))
    
    # change state to next state, so we are done with an episode
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  
  # Save the model every 10 episodes
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/252 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 96.190002
AI Trader sold:  $ 108.937500  Profit: $ 12.747498
AI Trader bought:  $ 109.665001
AI Trader bought:  $ 110.062500
AI Trader bought:  $ 113.902496
AI Trader sold:  $ 111.112503  Profit: $ 1.447502
AI Trader sold:  $ 112.727501  Profit: $ 2.665001
AI Trader sold:  $ 114.607498  Profit: $ 0.705002
AI Trader bought:  $ 124.370003
AI Trader sold:  $ 125.857498  Profit: $ 1.487495
AI Trader bought:  $ 126.522499
AI Trader sold:  $ 125.010002  Profit: - $ 1.512497
AI Trader bought:  $ 117.320000
AI Trader sold:  $ 113.489998  Profit: - $ 3.830002


 13%|█▎        | 34/252 [00:08<04:15,  1.17s/it]

AI Trader bought:  $ 115.360001


 14%|█▍        | 35/252 [00:12<06:36,  1.83s/it]

...

AI Trader bought:  $ 146.389999


 98%|█████████▊| 246/252 [12:17<00:19,  3.23s/it]

AI Trader bought:  $ 142.449997


 98%|█████████▊| 247/252 [12:20<00:16,  3.23s/it]

AI Trader bought:  $ 146.149994


In [ ]:
Test?